<strong>주가 예측 프로젝트 prac 1</strong>

- fdr 라이브러리 사용
- 프로그래밍적 사고
- pandas dataframe 자료구조는 열 기반 자료 구조로 반복적으로 처리하는 사고력을 기르기 부적합
- list 형태로 바꿔서 사용

<strong> 수행시간 O(N)</strong> 
- 과제 1: 0초
- 과제 2: 4분 50초
- 과제 3: 4분

# 과제 1
- tab separated 형식
- fdr 라이브러리를 사용하여 전체 종목 코드 다운받기
- 상장일 2018년 1월 2일 이후 종목 텍스트 파일로 저장하기<br>

In [1]:
import pandas as pd
import numpy as np
import FinanceDataReader as fdr
from tqdm import tqdm

In [2]:
df_stock = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download', header=0)[0] #데이터 불러오기
df_stock.head()

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,BNK금융지주,138930,기타 금융업,금융지주회사,2011-03-30,12월,빈대인,http://www.bnkfg.com,부산광역시
1,DRB동일,4840,고무제품 제조업,"고무벨트(V벨트,콘베이어벨트,평벨트),프라스틱제품 제조,판매",1976-05-21,12월,류영식,http://drbworld.com,부산광역시
2,DSR,155660,1차 비철금속 제조업,합섬섬유로프,2013-05-15,12월,홍석빈,http://www.dsr.com,부산광역시
3,GS,78930,기타 금융업,지주회사/부동산 임대,2004-08-05,12월,"허태수, 홍순기 (각자 대표이사)",NaN,서울특별시
4,HDC현대산업개발,294870,건물 건설업,"외주주택, 자체공사, 일반건축, 토목 등",2018-06-12,12월,"최익훈, 정익희, 김회언 (각자 대표이사)",http://www.hdc-dvp.com,서울특별시


In [3]:
lst_stock = df_stock.values.tolist() #데이터를 dataframe에서 리스트로 변환

In [4]:
lst_result = []


for row in tqdm(lst_stock):
    code, name, date = str(row[1]).zfill(6), row[0] , str(row[4])
    if date <= '2018-01-02' : 
            lst_result.append([code,name])

OF = open('../../data/assingnment1.txt','w')
for row in lst_result:
    OF.write(row[0]+'\t'+row[1]+'\n')
OF.close()

100%|██████████████████████████████████████████████████████████████████████████| 2586/2586 [00:00<00:00, 216259.00it/s]


# 과제 2

[참고]https://financedata.github.io/posts/finance-data-reader-users-guide.html
- 과제1에서 선정한 종목 중에서 거래대금 1조 이상인 날짜 수집
- Date, Open, High, Close, Volume, Change

In [5]:
lst_code2date = []

start_date = '20180102'
end_date = '20211231'


for row in tqdm(lst_result):
    code = row[0]
    stock = fdr.DataReader(code, start_date ,end_date)
    stock.index = stock.index.strftime('%Y-%m-%d')
    stock = stock.reset_index('Date')
    lst_stock = stock.values.tolist()
    for row in lst_stock:
        date, close, volume, trading = row[0], row[3],row[4], row[3]*row[4]
        if trading >= 1000000000000:
            lst_code2date.append([code, date])

100%|██████████████████████████████████████████████████████████████████████████████| 1961/1961 [04:50<00:00,  6.75it/s]


# 과제 3
- 머신러닝 데이터 만들기
- 과제2에서 선정한 거래대금 1조 이상인 종목과 날자를 기준일로하여 거래일을 기준으로하여
기준일 포함 10일 간 주가와 거래대금을 설명변쑤로 기준일 다음날의 2% 상승여부를 반응변수로 하는 데이터를 만듦

In [6]:
lst_result = []


OF = open('../../data/assignment3.txt', 'w')
for row in tqdm(lst_code2date):
    code, D0_date = row[0], row[1]
    
    stock = fdr.DataReader(code, start= '20171215' ,end = '20220115')
    stock.index = stock.index.strftime('%Y-%m-%d')
    stock = stock.reset_index('Date')
    lst_stock = stock.values.tolist()
    
    for i,row in enumerate(lst_stock):
        date = row[0]
        if i < 9 :
            continue
        if date not in D0_date:
            continue
        
        sub_stock = lst_stock[i-9:i+1]
        pred_price = lst_stock[i+1][-1]
        result = []
        
        for j,row2 in enumerate(sub_stock):
            open, high, low, close, volume = row2[1:6]
            trading_value = close*volume
            result += [open, high, low, close, trading_value]

        result = ','.join(map(str,result))
        ratio = str(int(lst_stock[i+1][-1]>=0.02))
        
        a=('\t'.join(map(str,[code, date, result, ratio]))+ '\n')

        OF.write('\t'.join(map(str,[code, date, result, ratio]))+ '\n')
OF.close()

100%|██████████████████████████████████████████████████████████████████████████████| 1666/1666 [04:02<00:00,  6.87it/s]
